# CopasiProcess Composer

In [1]:
import sys 

sys.path.insert(0, '..')

In [2]:
!python3 --version

Python 3.10.13


In [3]:
from basico import *
from basico import biomodels
from process_bigraph import Composite, pf 
from biosimulator_processes.processes.copasi_process import CopasiProcess

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


## Example 1: Define a single process instance and a corresponding emitter

##### 1. Define the schema by which our Composite instance will be configured

In [5]:
instance = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model_file': 'biosimulator_processes/model_files/Caravagna2010.xml'
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    },
    # TODO: Add emitter schema
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    }
}

In [7]:
def fetch_biomodel(term: str, index: int = 0):
        """Search for models matching the term and return an instantiated model from BioModels.

            Args:
                term:`str`: search term
                index:`int`: selector index for model choice

            Returns:
                `CDataModel` instance of loaded model.
            TODO: Implement a dynamic search of this
        """
        models = biomodels.search_for_model(term)
        model = models[index]
        sbml = biomodels.get_content_for_model(model['id'])
        return load_model_from_string(sbml)

In [8]:
model = fetch_biomodel('insulin')

In [9]:
get_reactions(model=model)

,scheme,flux,particle_flux,function,key,sbml_id,mapping
name,,,,,,,
v1,EGF + EGFR = EGF-EGFR,744300.0,744300.0,Function for v1,Reaction_0,v1,"{'c1': 'extracellular volume', 'c2': 'cytoplas..."
v2,2 * EGF-EGFR = EGF-EGFR^2,0.0,0.0,Function for v2,Reaction_1,v2,"{'c2': 'cytoplasm', 'k2': 'k2', 'kr2': 'kr2', ..."
v3,EGF-EGFR^2 = EGF-EGFR*^2,0.0,0.0,Function for v3,Reaction_2,v3,"{'c2': 'cytoplasm', 'k3': 'k3', 'kr3': 'kr3', ..."
v4,EGF-EGFR*^2-GAP-Grb2 + Prot = EGF-EGFR*^2-GAP-...,0.0,0.0,Function for v4,Reaction_3,v4,"{'c2': 'cytoplasm', 'k4': 'k4', 'kr4': 'kr4', ..."
v5,EGF-EGFR*^2-GAP-Grb2-Prot = EGF-EGFRi*^2-GAP-G...,0.0,0.0,Function for v5,Reaction_4,v5,"{'k5': 'k5', 'x7': 'EGF-EGFR*^2-GAP-Grb2-Prot'}"
...,...,...,...,...,...,...,...
v121,EGF-EGFR*^2-GAP-Shc*-Grb2-Sos-Ras-GDP + Prot =...,0.0,0.0,Function for v121,Reaction_120,v121,"{'c2': 'cytoplasm', 'k4': 'k4', 'kr4': 'kr4', ..."
v122,EGF-EGFR*^2-GAP-Shc*-Grb2-Sos-Ras-GDP-Prot = P...,0.0,0.0,Function for v122,Reaction_121,v122,"{'k5': 'k5', 'x93': 'EGF-EGFR*^2-GAP-Shc*-Grb2..."
v123,EGF-EGFR*^2-GAP-Shc*-Grb2-Sos-Ras-GTP = EGF-EG...,0.0,0.0,Function for v123,Reaction_122,v123,"{'c2': 'cytoplasm', 'k6': 'k6', 'kr6': 'kr6', ..."


In [10]:
config = {
    'reactions': {
        'R1': {
            'scheme': 'A -> B'
        }
    }
}

In [11]:
for reaction_name, reaction_spec in config['reactions'].items():
                    add_reaction(
                        name=reaction_name,
                        scheme=reaction_spec['scheme']
                    )

In [12]:
get_reactions(model=model)

,scheme,flux,particle_flux,function,key,sbml_id,mapping
name,,,,,,,
v1,EGF + EGFR = EGF-EGFR,744300.0,744300.0,Function for v1,Reaction_0,v1,"{'c1': 'extracellular volume', 'c2': 'cytoplas..."
v2,2 * EGF-EGFR = EGF-EGFR^2,0.0,0.0,Function for v2,Reaction_1,v2,"{'c2': 'cytoplasm', 'k2': 'k2', 'kr2': 'kr2', ..."
v3,EGF-EGFR^2 = EGF-EGFR*^2,0.0,0.0,Function for v3,Reaction_2,v3,"{'c2': 'cytoplasm', 'k3': 'k3', 'kr3': 'kr3', ..."
v4,EGF-EGFR*^2-GAP-Grb2 + Prot = EGF-EGFR*^2-GAP-...,0.0,0.0,Function for v4,Reaction_3,v4,"{'c2': 'cytoplasm', 'k4': 'k4', 'kr4': 'kr4', ..."
v5,EGF-EGFR*^2-GAP-Grb2-Prot = EGF-EGFRi*^2-GAP-G...,0.0,0.0,Function for v5,Reaction_4,v5,"{'k5': 'k5', 'x7': 'EGF-EGFR*^2-GAP-Grb2-Prot'}"
...,...,...,...,...,...,...,...
v122,EGF-EGFR*^2-GAP-Shc*-Grb2-Sos-Ras-GDP-Prot = P...,0.0,0.0,Function for v122,Reaction_121,v122,"{'k5': 'k5', 'x93': 'EGF-EGFR*^2-GAP-Shc*-Grb2..."
v123,EGF-EGFR*^2-GAP-Shc*-Grb2-Sos-Ras-GTP = EGF-EG...,0.0,0.0,Function for v123,Reaction_122,v123,"{'c2': 'cytoplasm', 'k6': 'k6', 'kr6': 'kr6', ..."
v124,EGF-EGFR*^2-GAP-Shc*-Grb2-Sos-Ras-GTP + Prot =...,0.0,0.0,Function for v124,Reaction_123,v124,"{'c2': 'cytoplasm', 'k4': 'k4', 'kr4': 'kr4', ..."


##### 2. Instantiate the Process-Bigraph-Engine with the Composite class

In [6]:
workflow = Composite({
    'state': instance
})

There is no model, creating a new one


AttributeError: 'NoneType' object has no attribute 'index'

##### 3. Run the Composite workflow with the Engine

In [6]:
workflow.run(10)

NameError: name 'workflow' is not defined

##### 4. Gather and neatly-print the results.

In [7]:
results = workflow.gather_results()

print(f'RESULTS: {pf(results)}')

NameError: name 'workflow' is not defined

##### 5. Plot/Analyze the results with Plotly

In [8]:
# 5. Plot/Analyze the results

### Example 2: Define multiple processes in the same instance for a parameter scan

In [8]:
def fetch_biomodel(term: str, index: int = 0):
        """Search for models matching the term and return an instantiated model from BioModels.

            Args:
                term:`str`: search term
                index:`int`: selector index for model choice

            Returns:
                `CDataModel` instance of loaded model.
            TODO: Implement a dynamic search of this
        """
        models = biomodels.search_for_model(term)
        model = models[index]
        sbml = biomodels.get_content_for_model(model['id'])
        return load_model_from_string(sbml)

In [9]:
model = fetch_biomodel('autism')

In [20]:
def fetch_biomodel(term: str, index: int = 0):
        """Search for models matching the term and return an instantiated model from BioModels.

            Args:
                term:`str`: search term
                index:`int`: selector index for model choice

            Returns:
                `CDataModel` instance of loaded model.
            TODO: Implement a dynamic search of this
        """
        models = biomodels.search_for_model(term)
        model = models[index]
        sbml = biomodels.get_content_for_model(model['id'])
        sbml_string = load_model_from_string(sbml)
        return sbml_string

In [21]:
model = fetch_biomodel('autism')

In [22]:
model

<CDataModel "Root">

In [30]:
tc = run_time_course(duration = 100)

In [31]:
tc.plot();

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/alex/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-KVuYbFzt-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/s3/2mt567997bj0_z5_19sd89500000gn/T/ipykernel_8199/2939347939.py", line 1, in <module>
    tc.plot();
    ^^^^^^^^^
  File "/Users/alex/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-KVuYbFzt-py3.11/lib/python3.11/site-packages/pandas/plotting/_core.py", line 947, in __call__
  File "/Users/alex/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-KVuYbFzt-py3.11/lib/python3.11/site-packages/pandas/plotting/_core.py", line 1944, in _get_plot_backend
  File "/Users/alex/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-KVuYbFzt-py3.11/lib/python3.11/site-packages/pandas/plotting/_core.py", line 1874, in _load_backend
ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

During han